In [2]:
import pandas as pd

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
cus = pd.read_csv('files_for_lab/customer_churn.csv')
cus['Churn'].value_counts(dropna=False)

No     5174
Yes    1869
Name: Churn, dtype: int64

In [5]:
data = cus[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'Churn']]
data

,tenure,SeniorCitizen,MonthlyCharges,Churn
0,1,0,29.85,No
1,34,0,56.95,No
2,2,0,53.85,Yes
3,45,0,42.30,No
4,2,0,70.70,Yes
...,...,...,...,...
7038,24,0,84.80,No
7039,72,0,103.20,No
7040,11,0,29.60,No
7041,4,1,74.40,Yes


In [9]:
y = data['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
y = pd.DataFrame(y)
y

,Churn
0,0
1,0
2,1
3,0
4,1
...,...
7038,0
7039,0
7040,0
7041,1


In [10]:
# i decided to leave the SeniorCitizen column as a numerical because minmax of 0 and 1 ist still 0 and 1

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

X = data.drop(['Churn'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = MinMaxScaler().fit(X_train)
X_train_norm = scaler.transform(X_train)
X_train_norm = pd.DataFrame(X_train_norm,columns=X_train.columns)

X_test_norm = scaler.transform(X_test)
X_test_norm = pd.DataFrame(X_test_norm,columns=X_train.columns)

y_train = y_train.reset_index(drop=True)
y_test= y_test.reset_index(drop=True)


In [11]:
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(random_state=42, solver='saga',
                  multi_class='multinomial').fit(X_train, y_train)

print(classification.score(X_train_norm, y_train))
print(classification.score(X_test_norm, y_test))

0.7368421052631579
0.7279954571266326


/home/mabioo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/mabioo/.local/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [12]:
# high acc score because big mayority in NO, therefore easy to get high score by NO-prediction, BUT many false negatives! 
# ---> customers will churn unexpected
# that manifests in very low recall, f1

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

predictions = classification.predict(X_test)

print(confusion_matrix(y_test, predictions))
print("precision:\t",precision_score(y_test, predictions))
print("recall:\t\t",recall_score(y_test, predictions))
print("f1:\t\t",f1_score(y_test, predictions))

[[1186   96]
 [ 257  222]]
precision:	 0.6981132075471698
recall:		 0.46346555323590816
f1:		 0.5570890840652447


In [13]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42, k_neighbors=3)
X_train_SMOTE,y_train_SMOTE = sm.fit_resample(X_train_norm, y_train)
y_train_SMOTE.value_counts()

Churn
0        3892
1        3892
dtype: int64

In [14]:
clf = LogisticRegression(random_state=42, solver='saga',
                  multi_class='multinomial').fit(X_train_SMOTE, y_train_SMOTE)

print(clf.score(X_train_SMOTE, y_train_SMOTE))
print(clf.score(X_test_norm, y_test))

0.7272610483042138
0.7416240772288473


/home/mabioo/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
preds = clf.predict(X_test)

print(confusion_matrix(y_test, preds))
print("precision:\t",precision_score(y_test, preds))
print("recall:\t\t",recall_score(y_test, preds))
print("f1:\t\t",f1_score(y_test, preds))

### much higher recall so no false negatives (customers that will churn unexpected)

[[424 858]
 [ 12 467]]
precision:	 0.35245283018867923
recall:		 0.9749478079331941
f1:		 0.5177383592017738
